In [10]:
import sys
!{sys.executable} -m pip install sagemaker-experiments

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial
from smexperiments.trial_component import TrialComponent

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


In [17]:
def cleanup_sme_sdk(experiment):
    for trial_summary in experiment.list_trials():
        trial = Trial.load(trial_name=trial_summary.trial_name)
        for trial_component_summary in trial.list_trial_components():
            tc = TrialComponent.load(
                trial_component_name=trial_component_summary.trial_component_name)
            trial.remove_trial_component(tc)
            try:
                # comment out to keep trial components
                tc.delete()
            except:
                # tc is associated with another trial
                continue
            # to prevent throttling
            time.sleep(.5)
        trial.delete()
        experiment_name = experiment.experiment_name
    experiment.delete()
    print(f"\nExperiment {experiment_name} deleted")


In [18]:
for i in Experiment.list():
    print(i.experiment_name)
    cleanup_sme_sdk(Experiment.load(i.experiment_name))

In [19]:
for i in TrialComponent.list():
    a = TrialComponent.load(i.trial_component_name)
    a.delete()
    print(i.trial_component_name)

In [20]:
import time
import os
from sagemaker import get_execution_role, session
import boto3

region = boto3.Session().region_name

role = get_execution_role()

sm_client = boto3.client('sagemaker', region_name=region)


In [21]:
model_groups = sm_client.list_model_package_groups()
for i in model_groups['ModelPackageGroupSummaryList']:
    print(i['ModelPackageGroupName'])
    model_packages = sm_client.list_model_packages(ModelPackageGroupName=i['ModelPackageGroupName'])
    for j in model_packages['ModelPackageSummaryList']:
        sm_client.delete_model_package(ModelPackageName=j['ModelPackageArn'])
    sm_client.delete_model_package_group(ModelPackageGroupName=i['ModelPackageGroupName'])

In [23]:
sm_client.delete_pipeline(PipelineName='sm-mnist-pipeline')
sm_client.delete_pipeline(PipelineName='sm-mnist-pipeline-notebook')
sm_client.delete_pipeline(PipelineName='sm-mnist-pipeline-notebook-hpo')

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:238312515155:pipeline/sm-mnist-pipeline-notebook',
 'ResponseMetadata': {'RequestId': '7fbd4c87-6361-400a-a7c0-12baca959fba',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7fbd4c87-6361-400a-a7c0-12baca959fba',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '99',
   'date': 'Wed, 09 Feb 2022 16:51:26 GMT'},
  'RetryAttempts': 0}}